# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [59]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df= pd.read_csv("assets/nhl.csv")
cities= pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nhl_df= pd.read_csv("assets/nhl.csv")
    nhl_df=nhl_df[~nhl_df['team'].str.contains('Division')]
    nhl_df= nhl_df[nhl_df['year']==2018].dropna()
    #alternative to : df[df['team']=='New York Islanders']= 'new_name'
    nhl_df.loc[(nhl_df['team'] == 'New York Islanders'),'team']='RangersIslandersDevils' 
    nhl_df.loc[(nhl_df['team'] == 'New Jersey Devils*'),'team']='RangersIslandersDevils'
    nhl_df.loc[(nhl_df['team'] == 'New York Rangers'),'team']='RangersIslandersDevils'
    nhl_df.loc[(nhl_df['team'] == 'Anaheim Ducks*'),'team']='KingsDucks'
    nhl_df.loc[(nhl_df['team'] == 'Los Angeles Kings*'),'team']='KingsDucks'
    
    cities_nhl=cities.copy()
    cities_nhl = cities_nhl.replace(to_replace ='(\[.*\])', value = '', regex = True)
    indexNames = cities_nhl[(cities_nhl['NHL']=='—') | (cities_nhl['NHL']=='')].index
    cities_nhl.drop(indexNames, inplace=True)

    win=[];loss=[];team=[];metro=[];pop=[]
    for i,l,m in zip(nhl_df['team'],nhl_df['W'],nhl_df['L']):
        for j,k,p in zip(cities_nhl['NHL'],cities_nhl['Metropolitan area'],cities_nhl['Population (2016 est.)[8]']):
            if j in i:
                win.append(l)
                loss.append(m)
                team.append(j)
                metro.append(k)
                pop.append(p)
    df=pd.DataFrame(columns=['team', 'metro', 'win', 'loss', 'pop'])
    df['win']= pd.Series(win)
    df['loss']= pd.Series(loss)
    df['team']= pd.Series(team)
    df['metro']= pd.Series(metro)
    df['pop']= pd.Series(pop)
    
    df['win']=df['win'].astype(int)
    df['loss']=df['loss'].astype(int)
    df['Ratio']= df['win']/(df['win']+df['loss'])
    df['pop']=df['pop'].astype(int)
    
    df_result= pd.DataFrame(df.groupby(['team']).mean())
    df_result=df_result.reset_index()
    
    population_by_region = [i for i in df_result['pop']] # pass in metropolitan area population from cities   
    win_loss_by_region = [i for i in df_result['Ratio']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"  
    assert len(population_by_region) == 28,   "Q1: There should be 28 teams being analysed for NHL"    
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [98]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities= pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df=nba_df[~nba_df['team'].str.contains('Division')]
    nba_df= nba_df[nba_df['year']==2018].dropna()
    nba_df.loc[(nba_df['team'] == 'New York Knicks\xa0(11)'),'team']='KnicksNets' 
    nba_df.loc[(nba_df['team'] == 'Brooklyn Nets\xa0(12)'),'team']='KnicksNets' 
    nba_df.loc[(nba_df['team'] == 'Los Angeles Clippers\xa0(10)'),'team']='LakersClippers' 
    nba_df.loc[(nba_df['team'] == 'Los Angeles Lakers\xa0(11)'),'team']='LakersClippers' 
    
    cities_nba=cities.copy()
    cities_nba = cities_nba.replace(to_replace ='(\[.*\])', value = '', regex = True)
    indexNames = cities_nba[(cities_nba['NBA']=='—') | (cities_nba['NBA']=='')].index
    cities_nba.drop(indexNames, inplace=True)

    win=[];loss=[];team=[];metro=[];pop=[]
    for i,l,m in zip(nba_df['team'],nba_df['W'],nba_df['L']):
        for j,k,p in zip(cities_nba['NBA'],cities_nba['Metropolitan area'],cities_nba['Population (2016 est.)[8]']):
            if j in i:
                win.append(l)
                loss.append(m)
                team.append(j)
                metro.append(k)
                pop.append(p)
    df=pd.DataFrame(columns=['team', 'metro', 'win', 'loss', 'pop'])
    df['win']= pd.Series(win)
    df['loss']= pd.Series(loss)
    df['team']= pd.Series(team)
    df['metro']= pd.Series(metro)
    df['pop']= pd.Series(pop)
    
    df['win']=df['win'].astype(int)
    df['loss']=df['loss'].astype(int)
    df['Ratio']= df['win']/(df['win']+df['loss'])
    df['pop']=df['pop'].astype(int)
    
    df_result= pd.DataFrame(df.groupby(['team']).mean())
    df_result=df_result.reset_index()
    
    population_by_region = [i for i in df_result['pop']] # pass in metropolitan area population from cities   
    win_loss_by_region = [i for i in df_result['Ratio']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"  
    assert len(population_by_region) == 28,   "Q1: There should be 28 teams being analysed for NHL"    
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [127]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/mlb.csv")
cities= pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df=mlb_df[~mlb_df['team'].str.contains('Division')]
    mlb_df= mlb_df[mlb_df['year']==2018].dropna()
    mlb_df.loc[(mlb_df['team'] == 'Los Angeles Angels'),'team']='DodgersAngels'
    mlb_df.loc[(mlb_df['team'] == 'Los Angeles Dodgers'),'team']='DodgersAngels'
    mlb_df.loc[(mlb_df['team'] == 'New York Yankees'),'team']='YankeesMets'
    mlb_df.loc[(mlb_df['team'] == 'New York Mets'),'team']='YankeesMets'
    mlb_df.loc[(mlb_df['team'] == 'San Francisco Giants'),'team']='GiantsAthletics'
    mlb_df.loc[(mlb_df['team'] == 'Oakland Athletics'),'team']='GiantsAthletics'  
    mlb_df.loc[(mlb_df['team'] == 'Chicago White Sox'),'team']='CubsWhite Sox'
    mlb_df.loc[(mlb_df['team'] == 'Chicago Cubs'),'team']='CubsWhite Sox' 
    
    cities_mlb=cities.copy()
    cities_mlb = cities_mlb.replace(to_replace ='(\[.*\])', value = '', regex = True)
    indexNames = cities_mlb[(cities_mlb['MLB']=='—') | (cities_mlb['MLB']=='')].index
    cities_mlb.drop(indexNames, inplace=True)

    win=[];loss=[];team=[];metro=[];pop=[]
    for i,l,m in zip(mlb_df['team'],mlb_df['W'],mlb_df['L']):
        for j,k,p in zip(cities_mlb['MLB'],cities_mlb['Metropolitan area'],cities_mlb['Population (2016 est.)[8]']):
            if j in i:
                win.append(l)
                loss.append(m)
                team.append(j)
                metro.append(k)
                pop.append(p)
    df=pd.DataFrame(columns=['team', 'metro', 'win', 'loss', 'pop'])
    df['win']= pd.Series(win)
    df['loss']= pd.Series(loss)
    df['team']= pd.Series(team)
    df['metro']= pd.Series(metro)
    df['pop']= pd.Series(pop)
    
    df['win']=df['win'].astype(int)
    df['loss']=df['loss'].astype(int)
    df['Ratio']= df['win']/(df['win']+df['loss'])
    df['pop']=df['pop'].astype(int)
    
    df_result= pd.DataFrame(df.groupby(['team']).mean())
    df_result=df_result.reset_index()
    
    population_by_region = [i for i in df_result['pop']] # pass in metropolitan area population from cities   
    win_loss_by_region = [i for i in df_result['Ratio']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"  
    assert len(population_by_region) == 26,   "Q1: There should be 28 teams being analysed for NHL"    
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [164]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities= pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nfl_df=pd.read_csv("assets/nfl.csv")
    nfl_df=nfl_df[~nfl_df['team'].str.contains('AFC') & ~nfl_df['team'].str.contains('NFC')]
    nfl_df= nfl_df[nfl_df['year']==2018].dropna()
    nfl_df.loc[(nfl_df['team'] == 'New York Giants'),'team']='GiantsJets'
    nfl_df.loc[(nfl_df['team'] == 'New York Jets'),'team']='GiantsJets'
    nfl_df.loc[(nfl_df['team'] == 'Los Angeles Rams*'),'team']='RamsChargers'
    nfl_df.loc[(nfl_df['team'] == 'Los Angeles Chargers+'),'team']='RamsChargers'
    nfl_df.loc[(nfl_df['team'] == 'San Francisco 49ers'),'team']='49ersRaiders'
    nfl_df.loc[(nfl_df['team'] == 'Oakland Raiders'),'team']='49ersRaiders' 
    
    cities_nfl=cities.copy()
    cities_nfl = cities_nfl.replace(to_replace ='(\[.*\])', value = '', regex = True)
    indexNames = cities_nfl[(cities_nfl['NFL']=='—') | (cities_nfl['NFL']=='') | (cities_nfl['NFL']=='— ')].index
    cities_nfl.drop(indexNames, inplace=True)

    win=[];loss=[];team=[];metro=[];pop=[]
    for i,l,m in zip(nfl_df['team'],nfl_df['W'],nfl_df['L']):
        for j,k,p in zip(cities_nfl['NFL'],cities_nfl['Metropolitan area'],cities_nfl['Population (2016 est.)[8]']):
            if j in i:
                win.append(l)
                loss.append(m)
                team.append(j)
                metro.append(k)
                pop.append(p)
    df=pd.DataFrame(columns=['team', 'metro', 'win', 'loss', 'pop'])
    df['win']= pd.Series(win)
    df['loss']= pd.Series(loss)
    df['team']= pd.Series(team)
    df['metro']= pd.Series(metro)
    df['pop']= pd.Series(pop)
    
    df['win']=df['win'].astype(int)
    df['loss']=df['loss'].astype(int)
    df['Ratio']= df['win']/(df['win']+df['loss'])
    df['pop']=df['pop'].astype(int)
    
    df_result= pd.DataFrame(df.groupby(['team']).mean())
    df_result=df_result.reset_index()
    
    population_by_region = [i for i in df_result['pop']] # pass in metropolitan area population from cities   
    win_loss_by_region = [i for i in df_result['Ratio']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"  
    assert len(population_by_region) == 29,   "Q1: There should be 28 teams being analysed for NHL"    
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values